In [26]:
import requests
import base64
import pandas as pd
from pprint import pprint
import json
from urllib.parse import urlencode
from flask import Flask, render_template, request, redirect, url_for,jsonify

# Get TOKEN


In [27]:
def getToken():
    client_id = "e3aaa86dda014fd3b0c7d0df640cf422"
    client_secret = "28fe6be9003c45b897164682d1f83751"
    client_creds = f"{client_id}:{client_secret}"
    client_creds_64 = base64.b64encode(client_creds.encode())
    token_url = "https://accounts.spotify.com/api/token"
    method = "POST"
    token_data = {
        "grant_type": "client_credentials"
    }
    token_header = {
        "Authorization": f"Basic {client_creds_64.decode()}"
    }
    r = requests.post(token_url, data=token_data, headers=token_header)
    response = r.json()
    access_token = response['access_token']
    head = {
     "Authorization": f"Bearer {access_token}"
    }
    return head

# Get playlist by Id

In [28]:
def get_playlist_by_id(id):
    playlist_id = id
    BASE_URL = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    

    response = requests.request("GET", BASE_URL, headers=getToken())
    return response.json()

#pprint(get_playlist_by_id('2wTyHjihNbUovf1uRE4GMh'))




# Get top Playlist by Categorie


In [29]:
playlist_cat = "https://api.spotify.com/v1/search"
def topPlaylist(categorie, limit = 10):
    resultat_final = []
    
    for cat in categorie:
        playlist_clean = []
        dict_par_categorie = {}
        
        params = urlencode({"q": cat, "type": "playlist", "limit": limit})
        playlist_url = f"{playlist_cat}?{params}"
        response = requests.get(playlist_url, headers=getToken())
        #Si tous s'est bien passé
        if response.status_code == 200:
            list_playlists = response.json()['playlists']['items']
            
            for p in list_playlists:
                keys = ['name', 'description', 'id']
                playlist = {k:p[k] for k in keys}
                playlist_clean.append(playlist)
            dict_par_categorie[cat] = playlist_clean
            resultat_final.append(dict_par_categorie)
    return resultat_final

#topPlaylist(['sport','motivation'],10)

# Get artist by id 

In [30]:
artist_id = '3fMbdgg4jU18AjLCKBhRSm'


def get_artist_by_id(id):
    artist_id = id
    BASE_URL = f"https://api.spotify.com/v1/artists/{artist_id}"
    

    response = requests.request("GET", BASE_URL, headers=getToken())
    return response.json()

#pprint(get_artist_by_id(artist_id))

# Get albums by artist_id

In [31]:
def get_albums_by_artist_id(id):
    artist_id = id
    BASE_URL = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    

    response = requests.request("GET", BASE_URL, headers=getToken())
    return response.json()

#pprint(get_albums_by_artist_id('3fMbdgg4jU18AjLCKBhRSm'))

In [32]:
with open('test_json.json','w') as f:
    json.dump(get_albums_by_artist_id('3fMbdgg4jU18AjLCKBhRSm'),f)

# Get nb followers by playlist


In [33]:
def nb_followers(playlist_id):
    response = get_playlist_by_id(playlist_id)['followers']
   
    return response

print(nb_followers('2wTyHjihNbUovf1uRE4GMh'))

{'href': None, 'total': 20409}


In [34]:

url_sadi= "https://api.spotify.com/v1/search"  

def get_albums_by_artist_per_country(artist_name,code_country,limit=10):

    params = urlencode({"q": artist_name,"market": code_country,"type":"album", "limit": limit})
    test = f"{url_sadi}?{params}"
    #print(test)
          
    response = requests.get(test, headers=getToken())
    if response.status_code == 200:
        list_albums=response.json()['albums']['items']
        albums_clean=[]
        for item in list_albums:
            keys=['name','external_urls']
            albums_dic={k:item[k] for k in keys}
            albums_clean.append(albums_dic)
        json_response = {'albums':albums_clean}
    return json_response

#pprint(get_albums_by_artist_per_country("jackson","Fr",10))
   


In [35]:
#une methode qui renvoi un top x de playlist selon une categorire
playlist_cat = "https://api.spotify.com/v1/search"
def topPlaylist(categorie, limit = 10):
    resultat_final = []
    
    for cat in categorie:
        playlist_clean = []
        dict_par_categorie = {}
        
        params = urlencode({"q": cat, "type": "playlist", "limit": limit})
        playlist_url = f"{playlist_cat}?{params}"
        response = requests.get(playlist_url, headers=getToken())
        #Si tous s'est bien passé
        if response.status_code == 200:
            list_playlists = response.json()['playlists']['items']
            
            for p in list_playlists:
                keys = ['name', 'description', 'id']
                playlist = {k:p[k] for k in keys}
                playlist_clean.append(playlist)
            dict_par_categorie[cat] = playlist_clean
            resultat_final.append(dict_par_categorie)
    return resultat_final

#pprint(topPlaylist(['sport','motivation'],2))


# get_nb_followers_by_playlist_name for top playlist

In [36]:
# def get_nb_followers_by_playlist_name(categorie, nom):
#     cat=[]
#     cat.append(categorie)

#     list_cat=topPlaylist(cat,10)

#     listtop = list_cat[0].get(cat[0])
#     nb_followers=-1
#     i=0
#     while i < len(listtop) :
#         if listtop[i].get('name') == nom:
#             nb_followers = get_nb_followers_by_playlist(listtop[i].get('id'))
#         i+=1
#     return nb_followers

# print(get_nb_followers_by_playlist_name('motivation','MOTIVATION'))
# #
# # print(get_nb_followers_by_playlist_name('motivation','Morning Motivation'))

## id playlist
- sport :
2Ikjjx2x7HX0Iw90AAKgXS
- motivation :
78Suoigk80gI9mI9z5tYl8

## id artist
1rT9J8vtPCzEKL7oGntxnr

In [38]:
app=Flask(__name__)

@app.route('/')
def home(): 
    return render_template("index.html")

@app.route('/categorie')
def categorie(): 
    response = topPlaylist(['sport','motivation','études','amour'])
    #pprint(response) 
    return jsonify(response)
    

@app.route('/artist')
def artist_by_id():
    id=request.args.get('id') 
    response = get_artist_by_id(id)
    #pprint(response) 
    return jsonify(response)


@app.route('/playlist')
def playlist_by_id(): 
    id=request.args.get('id')
    response = get_playlist_by_id(id)
    #pprint(response) 
    return jsonify(response)


@app.route('/albumsArtist')
def albums_by_artist_id(): 
    id=request.args.get('id') 
    response = get_albums_by_artist_id(id)
    #pprint(response) 
    return jsonify(response)

@app.route('/albumsCountry')
def albums_by_artist_per_country(): 
    name=request.args.get('name')
    print(name)
    country=request.args.get('country')
    print(country)
    response = get_albums_by_artist_per_country(name,country,10)
    #pprint(response)
    return jsonify(response)

@app.route('/followers')
def get_nb_followers():
    id=request.args.get('id') 
    response = nb_followers(id)
    #pprint(response)
    return jsonify(response)

    
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Oct/2021 16:06:50] "GET /playlist?id=2Ikjjx2x7HX0Iw90AAKgXS HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 16:06:56] "GET /categorie HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 16:07:02] "GET /playlist?id=2Ikjjx2x7HX0Iw90AAKgXS HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 16:07:07] "GET /artist?id=1rT9J8vtPCzEKL7oGntxnr HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 16:08:00] "GET /artist?id=1rT9J8vtPCzEKL7oGntxnr HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 16:08:07] "GET /albumsArtist?id=1rT9J8vtPCzEKL7oGntxnr HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 16:12:14] "GET /artist?id=1rT9J8vtPCzEKL7oGntxnr HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 16:12:17] "GET /albumsCountry?name=jackson&country=fr HTTP/1.1" 200 -


jackson
fr


127.0.0.1 - - [15/Oct/2021 16:12:20] "GET /followers?id=2wTyHjihNbUovf1uRE4GMh HTTP/1.1" 200 -
